# Embeddings: How Machines "Understand" Words

## Understand vs. Reading Text

安装依赖FOR GOOGLE COLAB11

In [3]:
!pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [4]:
!pip uninstall torch

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.5.1+cu121.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-2.5.1+cu121


In [6]:
pip install torch==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.21.5
    Uninstalling nvidia-nccl-cu12-2.21.5:
      Successfully uninstalled nvidia-nccl-cu12-2.21.5
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [7]:
pip install torchtext==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00


## Word Vectors

In [8]:
import torch
o = torch.zeros(20000).int()
o[1152] = 1

In [9]:
E = torch.nn.Embedding(num_embeddings = 20000, embedding_dim = 300)
e = E(o)

In [ ]:
e

### Word2Vec

### Embeddings in the Age of Transfer Learning

## Embeddings in Practice

### Preprocessing

In [12]:
import torch.nn.functional as F
import torch.nn as nn
from torch import optim

from torchtext import *
import torchtext

/usr/local/lib/python3.11/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.11/dist-packages/torchtext/nn/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.11/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is d

In [13]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [14]:
TEXT.build_vocab(train, vectors='glove.6B.100d')

NameError: name 'TEXT' is not defined

In [17]:
# set up fields
# TEXT = data.Field(lower=True, include_lengths=True, batch_first=False, tokenize='spacy')
TEXT = torchtext.legacy.data.Field(lower=True, include_lengths=True, batch_first=False, tokenize='spacy')

LABEL = data.LabelField()

# make splits for data
train, test = datasets.IMDB.splits(TEXT, LABEL)

# build the vocabulary
TEXT.build_vocab(train, vectors='glove.6B.100d') \
# use 'glove.42B.300d' for greater accuracy or \
#'glove.6B.100d' for greater speed
LABEL.build_vocab(train)

# make iterator for splits
train_iter, test_iter = data.BucketIterator.splits((train, test), \
batch_sizes=(128,1024), device=dev, sort_within_batch=True, repeat=False)

AttributeError: module 'torchtext.data' has no attribute 'Field'

### Model

In [ ]:
class RNN_classifier(nn.Module):
    def __init__(self, embedding_size = 100, hidden_size = 512, num_layers = 3):
        super().__init__()

        # Set up an embedding layer with the right dimensions, \
        and copy the weights from the pretrained glove embeddings
        vocab = TEXT.vocab
        self.embed = nn.Embedding(len(vocab), embedding_size).cuda()
        self.embed.weight.data.copy_(vocab.vectors)

        # Set up a standard PyTorch RNN sections with the right \
        dimensions and a variable number of layers
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)

        # Add a two layer classification head with the right dimensions. \
        The final layer must output a single number
        self.classificationLayer1 = nn.Linear(hidden_size,10)
        self.classificationLayer2 = nn.Linear(10,1)


    def forward(self, input, lengths=None):

        embed_input = self.embed(input)
        packed_emb = nn.utils.rnn.pack_padded_sequence(embed_input, \
        lengths, batch_first=False)

        output, hidden = self.rnn(packed_emb)
        hidden = hidden[-1]
        x = hidden.squeeze(0)
        x = self.classificationLayer1(x)
        x = self.classificationLayer2(x)

        logits = x.view(-1)
        return logits


In [ ]:
model = RNN_classifier(hidden_size=256, num_layers=1)
model.to(dev)

In [ ]:
for batch in train_iter:
    (x,x_len) = batch.text
    pred = model(x,x_len)
    print(pred.shape)
    break

### Training

In [ ]:
loss_func = F.binary_cross_entropy_with_logits
opt = optim.Adam(model.parameters(), lr=1e-4)
epochs = 6

In [ ]:
def get_metrics(model, test_data):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(test_data):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            total += batch_data.label.size(0)
            correct += (predicted_labels == batch_data.label.long()).sum()
        return correct.float()/total

In [ ]:
from tqdm import tqdm_notebook as tqdm

for epoch in tqdm(range(epochs)):
    model.train()
    for batch in tqdm(train_iter):
        (x,x_lengths)=batch.text
        pred = model(x,x_lengths)

        actual=batch.label.float()
        loss = loss_func(pred,actual)

        loss.backward()
        opt.step()
        opt.zero_grad()

    if (epoch==5):
        for g in opt.param_groups:
            g['lr'] = 3e-3

    print("Accuracy: " + str(get_metrics(model, test_iter).cpu().numpy()))

### Validation

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]

    tensor = torch.LongTensor(indexed).to(dev)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
review = """I like that Far From Home is trying something new and that its
humor  feels more real than the ironic cracks in most superhero movies.
I just wish its good pieces all came together more satisfyingly."""

print('Probability positive:')
predict_sentiment(model, review)

## Embedding Things That Aren't Words

#### A Sonnet in The MIDI Protocol

### Some General Tips for Making Custom Embeddings

## Conclusion